In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!ls /kaggle/input/vedai-dataset-coco

vedai_coco


In [3]:
import json
DATASET_PATH = '/kaggle/input/vedai-dataset-coco/vedai_coco'
test_data = {}
train_data = {}
with open(f'{DATASET_PATH}/train.json', 'r') as file:
    train_data = json.load(file)

with open(f'{DATASET_PATH}/test.json', 'r') as file:
    test_data = json.load(file)

In [4]:
classes = dict([(x['name'], x['id']-1) for x in train_data['categories']])
classes

{'car': 0,
 'truck': 1,
 'pickup': 2,
 'tractor': 3,
 'camping car': 4,
 'boat': 5,
 'motorcycle': 6,
 'bus': 7,
 'van': 8,
 'other': 9,
 'small car': 10,
 'large car': 11,
 'plane': 12,
 'board': 13}

In [5]:
print(f"{len(train_data['images'])} train images\n{len(test_data['images'])} test images") 

1999 train images
60 test images


In [6]:
!mkdir train
!mkdir val
!cp -r {DATASET_PATH}/test/* val
!cp -r {DATASET_PATH}/train/* train

In [7]:
train_image_df = pd.DataFrame(train_data['images'])
test_image_df = pd.DataFrame(test_data['images'])
train_image_df.head()

,id,file_name,width,height
0,1,5690.jpg,256,256
1,2,604.jpg,256,256
2,3,21117.jpg,256,256
3,4,610.jpg,256,256
4,5,176.jpg,256,256


In [8]:
train_annot_df = pd.DataFrame(train_data['annotations'])
test_annot_df = pd.DataFrame(test_data['annotations'])
train_annot_df.head()

,id,image_id,category_id,bbox,area,iscrowd
0,1,1,1,"[93, 28, 17, 40]",680,0
1,2,2,1,"[94, 3, 22, 29]",638,0
2,3,2,1,"[188, 141, 29, 20]",580,0
3,4,2,1,"[206, 187, 30, 18]",540,0
4,5,2,1,"[125, 218, 35, 18]",630,0


In [9]:
train_df = pd.merge(train_annot_df, train_image_df, how='left', left_on='image_id', right_on='id')
test_df = pd.merge(test_annot_df, test_image_df, how='left', left_on='image_id', right_on='id')
train_df.head()

,id_x,image_id,category_id,bbox,area,iscrowd,id_y,file_name,width,height
0,1,1,1,"[93, 28, 17, 40]",680,0,1,5690.jpg,256,256
1,2,2,1,"[94, 3, 22, 29]",638,0,2,604.jpg,256,256
2,3,2,1,"[188, 141, 29, 20]",580,0,2,604.jpg,256,256
3,4,2,1,"[206, 187, 30, 18]",540,0,2,604.jpg,256,256
4,5,2,1,"[125, 218, 35, 18]",630,0,2,604.jpg,256,256


In [10]:
# Convert Coco bb to Yolo
def coco_to_yolo(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

def bbox_converter(image):
    return coco_to_yolo(image.bbox[0], image.bbox[1], image.bbox[2], image.bbox[3], image.width, image.height)

def process_df(df):
    df['bbox_new'] = df.apply(bbox_converter, axis=1)
    df['class'] = df.category_id - 1
    return df

train_df = process_df(train_df)
test_df = process_df(test_df)
train_df.head()

,id_x,image_id,category_id,bbox,area,iscrowd,id_y,file_name,width,height,bbox_new,class
0,1,1,1,"[93, 28, 17, 40]",680,0,1,5690.jpg,256,256,"[0.396484375, 0.1875, 0.06640625, 0.15625]",0
1,2,2,1,"[94, 3, 22, 29]",638,0,2,604.jpg,256,256,"[0.41015625, 0.068359375, 0.0859375, 0.11328125]",0
2,3,2,1,"[188, 141, 29, 20]",580,0,2,604.jpg,256,256,"[0.791015625, 0.58984375, 0.11328125, 0.078125]",0
3,4,2,1,"[206, 187, 30, 18]",540,0,2,604.jpg,256,256,"[0.86328125, 0.765625, 0.1171875, 0.0703125]",0
4,5,2,1,"[125, 218, 35, 18]",630,0,2,604.jpg,256,256,"[0.556640625, 0.88671875, 0.13671875, 0.0703125]",0


In [11]:
def write_txt_labels(df, path):
    for image_id, images in df.groupby(by='image_id'):
        image_file = os.path.join(path, images.iloc[0].file_name)
        pre, ext = os.path.splitext(image_file)
        label_file = pre + '.txt'
        with open(label_file, 'w') as file:
            for ind, image in images.iterrows():
                file.write("{} {} {} {} {}\n".format(image['class'], image['bbox_new'][0], image['bbox_new'][1], image['bbox_new'][2], image['bbox_new'][3]))

In [12]:
write_txt_labels(test_df, './val')
!ls -l val | wc -l

121


In [13]:
write_txt_labels(train_df, './train')
!ls -l train | wc -l

3999


In [14]:
import yaml
print(yaml.dump({v: k for k, v in classes.items()}))

0: car
1: truck
2: pickup
3: tractor
4: camping car
5: boat
6: motorcycle
7: bus
8: van
9: other
10: small car
11: large car
12: plane
13: board



In [15]:
yaml_content = (
"""
path: /kaggle/working # dataset root dir
train: train # train images (relative to 'path') 4 images
val: val # val images (relative to 'path') 4 images
# no test images

# Classes (14 COCO classes)
names:
    0: car
    1: truck
    2: pickup
    3: tractor
    4: camping car
    5: boat
    6: motorcycle
    7: bus
    8: van
    9: other
    10: small car
    11: large car
    12: plane
    13: board
"""
)
with open('vedai.yaml', 'w') as file:
    file.write(yaml_content)

In [16]:
HOME = '.'
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

total 408M
-rw-r--r-- 1 root root  80M May 26 15:53 yolov10b.pt
-rw-r--r-- 1 root root 100M May 26 15:53 yolov10l.pt
-rw-r--r-- 1 root root  64M May 26 15:54 yolov10m.pt
-rw-r--r-- 1 root root  11M May 26 15:54 yolov10n.pt
-rw-r--r-- 1 root root  32M May 26 15:54 yolov10s.pt
-rw-r--r-- 1 root root 123M May 26 15:54 yolov10x.pt


In [17]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 17.8 MB/s eta 0:00:00


In [18]:
DATASET = '/kaggle/working/vedai.yaml'
!yolo task=detect mode=train epochs=300 batch=32 plots=True \
model={HOME}/weights/yolov10s.pt \
data={DATASET}

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=./weights/yolov10s.pt, data=/kaggle/working/vedai.yaml, epochs=300, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None,

In [19]:
!ls

__notebook__.ipynb  train	 val	    vedai.yaml	yolo11n.pt
runs		    train.cache  val.cache  weights
